## Zonotope type parameters

In [3]:
using Revise, LazySets, BenchmarkTools

In [2]:
c = rand(2)

2-element Array{Float64,1}:
 0.484108963306799
 0.679992862194166

In [3]:
gl = [rand(2) for _ in 1:100];

In [11]:
@btime Zonotope($c, $gl);
@btime Zonotope($c, $gl, remove_zero_generators=false);

  711.659 ns (2 allocations: 1.80 KiB)
  628.181 ns (2 allocations: 1.80 KiB)


In [10]:
@btime LazySets.Zonotope2($c, $gl);
@btime LazySets.Zonotope2($c, $gl, remove_zero_generators=false);

  709.309 ns (2 allocations: 1.80 KiB)
  586.228 ns (2 allocations: 1.80 KiB)


In [4]:
N = Float64
using Test

In [ ]:
# constructor from center and list of generators
function Zonotope(center::VN, generators_list::AbstractVector{VN};
                  remove_zero_generators::Bool=true) where {N<:Real, VN<:AbstractVector{N}}
    n = length(center)
    p = length(generators_list)
    G = Matrix{N}(undef, n, p) # TODO: generalize type, see #2040
    if remove_zero_generators
        for (i, gi) in enumerate(generators_list)
            iszero(gi) && continue
            @inbounds G[:, i] = gi
        end
    else
        for (i, gi) in enumerate(generators_list)
            @inbounds G[:, i] = gi
        end
    end
    return Zonotope(center, G; remove_zero_generators=false)
end

In [5]:
# conversion from zonotopic sets
Z = Zonotope(N[0, 0], hcat(N[1, 1]))
@test Z == convert(Zonotope, Z) && Z == togrep(Z)

Test Passed

In [6]:
AZ = Singleton(N[0, 0])

Singleton{Float64,Array{Float64,1}}([0.0, 0.0])

In [7]:
Z = Zonotope(center(AZ), genmat(AZ))

Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([0.0, 0.0], Array{Float64}(undef,2,0))

In [9]:
Z = convert(Zonotope, AZ)

Zonotope{Float64,Array{Float64,1},SubArray{Float64,2,LinearAlgebra.Diagonal{Float64,Array{Float64,1}},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false}}([0.0, 0.0], )

In [71]:
using LinearAlgebra

In [89]:
Zonotope(center(AZ), Diagonal(radius_hyperrectangle(AZ)), remove_zero_generators=false)

Zonotope{Float64,Array{Float64,1},Diagonal{Float64,Array{Float64,1}}}([0.0, 0.0], [0.0 0.0; 0.0 0.0])

In [91]:
z = Zonotope(center(AZ), Diagonal(radius_hyperrectangle(AZ)), remove_zero_generators=true)

Zonotope{Float64,Array{Float64,1},Diagonal{Float64,Array{Float64,1}}}([0.0, 0.0], )

In [11]:
size(Z.generators)

(2, 0)

In [26]:
@which linear_map(rand(2, 2), rand(Zonotope))

linear_map(M::AbstractArray{N,2}, Z::AbstractZonotope{N}) where N<:Real in LazySets at /home/mforets/.julia/dev/LazySets/src/Interfaces/AbstractZonotope.jl:339

In [95]:
D = Diagonal([1.0, 0.0, 2.0, 0.0, 5.0])

5×5 Diagonal{Float64,Array{Float64,1}}:
 1.0   ⋅    ⋅    ⋅    ⋅ 
  ⋅   0.0   ⋅    ⋅    ⋅ 
  ⋅    ⋅   2.0   ⋅    ⋅ 
  ⋅    ⋅    ⋅   0.0   ⋅ 
  ⋅    ⋅    ⋅    ⋅   5.0

In [88]:
Diagonal(filter(!iszero, diag(D)))

3×3 Diagonal{Float64,Array{Float64,1}}:
 1.0   ⋅    ⋅ 
  ⋅   2.0   ⋅ 
  ⋅    ⋅   5.0

In [57]:
function linear_map!(M, Z)
    c = Z.center
    G = Z.generators
    G .= G .* Z.generators
    c .= M * c
    return Z
end

linear_map! (generic function with 1 method)

In [58]:
M = rand(2, 2)

Z = rand(Zonotope, dim=2)

Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([1.8432068642298516, -0.2960448163188926], [-1.283912338361676 1.3225614346747865 -1.3740480994924142; 0.4147715750289514 0.679237803471378 -1.7122468464832161])

In [59]:
linear_map!(M, Z)

Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([1.034779572209615, 0.17748935844720737], [1.6484308925973468 1.7491687484890295 1.8880081797187156; 0.17203545945199705 0.4613639936646223 2.9317892632917184])

In [60]:
Z

Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([1.034779572209615, 0.17748935844720737], [1.6484308925973468 1.7491687484890295 1.8880081797187156; 0.17203545945199705 0.4613639936646223 2.9317892632917184])

In [61]:
@btime linear_map($M, $Z)

  352.144 ns (8 allocations: 672 bytes)


Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([0.6800617840512672, 0.23436819435034925], [1.0514227753233543 1.1960988689130492 1.9930518806817368; 0.32995673510992274 0.45941567116059934 1.4499266254450625])

  178.879 ns (1 allocation: 96 bytes)


Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([5.0e-324, 0.0], [Inf Inf Inf; 0.0 0.0 Inf])

In [18]:
convert(Zonotope, AZ) == togrep(AZ) == Z

MethodError: MethodError: Cannot `convert` an object of type SubArray{Float64,2,LinearAlgebra.Diagonal{Float64,Array{Float64,1}},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false} to an object of type LinearAlgebra.Diagonal{Float64,Array{Float64,1}}
Closest candidates are:
  convert(::Type{#s627} where #s627<:LinearAlgebra.Diagonal, !Matched::Union{LinearAlgebra.AbstractTriangular, LinearAlgebra.Bidiagonal, LinearAlgebra.Diagonal, LinearAlgebra.SymTridiagonal, LinearAlgebra.Tridiagonal}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/LinearAlgebra/src/special.jl:69
  convert(::Type{T}, !Matched::T) where T<:AbstractArray at abstractarray.jl:14
  convert(::Type{T}, !Matched::LinearAlgebra.Factorization) where T<:AbstractArray at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/LinearAlgebra/src/factorization.jl:53
  ...

In [9]:
togrep(AZ)

Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([0.0, 0.0], [-1.0; -1.0])

In [3]:
)
for AZ in [,
           ,
           ]
    Z = Zonotope(center(AZ), genmat(AZ))
    @test convert(Zonotope, AZ) == togrep(AZ) == Z
end

Error During Test at In[3]:8
  Test threw exception
  Expression: convert(Zonotope, AZ) == togrep(AZ) == Z
  MethodError: Cannot `convert` an object of type SubArray{Float64,2,LinearAlgebra.Diagonal{Float64,Array{Float64,1}},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false} to an object of type LinearAlgebra.Diagonal{Float64,Array{Float64,1}}
  Closest candidates are:
    convert(::Type{#s627} where #s627<:LinearAlgebra.Diagonal, !Matched::Union{LinearAlgebra.AbstractTriangular, LinearAlgebra.Bidiagonal, LinearAlgebra.Diagonal, LinearAlgebra.SymTridiagonal, LinearAlgebra.Tridiagonal}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/LinearAlgebra/src/special.jl:69
    convert(::Type{T}, !Matched::T) where T<:AbstractArray at abstractarray.jl:14
    convert(::Type{T}, !Matched::LinearAlgebra.Factorization) where T<:AbstractArray at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/LinearAlgebra/src/factorization.jl:53
    ...
  Stackt

Test.FallbackTestSetException: There was an error during testing

In [12]:
Z = rand(Zonotope, dim=1)

Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([-0.5945119887867256], [0.5073766335601305])

In [14]:
convert(Zonotope, CartesianProduct(Z, Z))

MethodError: MethodError: Cannot `convert` an object of type CartesianProduct{Float64,Zonotope{Float64,Array{Float64,1},Array{Float64,2}},Zonotope{Float64,Array{Float64,1},Array{Float64,2}}} to an object of type Zonotope
Closest candidates are:
  convert(::Type{Zonotope}, !Matched::Zonotope) at /home/mforets/.julia/dev/LazySets/src/convert.jl:8
  convert(::Type{Zonotope}, !Matched::AbstractHyperrectangle) at /home/mforets/.julia/dev/LazySets/src/convert.jl:259
  convert(::Type{Zonotope}, !Matched::AbstractZonotope) at /home/mforets/.julia/dev/LazySets/src/convert.jl:277
  ...

-----

## Contextual dispatch for LazySets

In [1]:
using Revise, LazySets, Cassette

In [2]:
using IntervalArithmetic

In [5]:
Cassette.@context FastPowersCtx
const fast_powers_ctx = Cassette.disablehooks(FastPowersCtx())

Cassette.overdub(::FastPowersCtx, ::typeof(^), a, b) = pow(a,b)

fast_powers(f) = Cassette.overdub(fast_powers_ctx, f)

fast_powers (generic function with 1 method)

In [ ]:
x = 1.1..2.2

fast_powers() do
    x^10
end

In [3]:
Cassette.@context ConcreteMinkowskiSumCtx
const concrete_minkowski_sum_ctx = Cassette.disablehooks(ConcreteMinkowskiSumCtx())

Cassette.overdub(::ConcreteMinkowskiSumCtx, ::typeof(+), X::LazySet, Y::LazySet) = LazySets.minkowski_sum(X, Y)

concrete_minkowski_sum(f) = Cassette.overdub(concrete_minkowski_sum_ctx, f)

concrete_minkowski_sum (generic function with 1 method)

In [4]:
rand(VPolygon)

VPolygon{Float64,Array{Float64,1}}(Array{Float64,1}[[-1.2691334565275214, -0.48447000232930226], [0.510488725170398, -1.3091771831391357], [2.8987858485690015, -0.07897111469423757], [0.26669087799980895, 0.00974954916428128], [-1.2503489390753801, -0.22090437985566158]])

In [5]:
X, Y = rand(VPolygon), rand(VPolygon)

(VPolygon{Float64,Array{Float64,1}}(Array{Float64,1}[[-2.891440369388445, 1.4974603483262503], [-2.7543914518160597, 1.0140102595539866], [-2.5160881915505673, 0.4341350873182388], [-1.8326871410944174, 0.2791562864595338], [-1.079324190333, 0.10959707549124292], [-0.5694666832207753, 0.02057642325639991], [0.9206192553020137, -0.18229943741611468], [0.690459131394798, 2.615053718175048], [-2.7090118148649993, 1.78797518892204]]), VPolygon{Float64,Array{Float64,1}}(Array{Float64,1}[[-1.189781063361007, 0.3256034667025641], [-1.1096833678107179, -0.10857070931184043], [-0.31982477529339226, -0.1128606495006116], [1.2135756215119895, -0.07501254151744109], [1.976518461380786, 0.5415029398039025], [2.6540623347530286, 1.3375271945122025], [2.331710899144571, 1.832837203302411], [1.3363698904893284, 2.455481645789108], [0.5346946762643379, 2.4834604152834245], [-1.0131794158530185, 1.0799080389885327]]))

In [8]:
concrete_minkowski_sum() do
    X, Y = rand(VPolygon), rand(VPolygon)
    X + Y
end

VPolygon{Float64,Array{Float64,1}}(Array{Float64,1}[[-4.387680426840783, 2.877203090673391], [-4.283301797865001, 1.5507669112713376], [-3.9294882548776684, 0.8879575633444368], [-2.524702191615023, -0.14636097004875492], [-1.447040692798065, -0.9102765557832327], [0.20823843174975198, -0.9152521796801377], [1.136630841164405, 0.48342955996494014], [1.645940898358691, 4.089699959892178], [1.6947413906818083, 6.17891182543696], [0.7595128744755058, 6.1592565854212875], [0.4767658258169887, 6.1454747613879785], [-1.0890240595361564, 5.984550698130458], [-1.9375600696326338, 5.818248887894724], [-2.324908858873632, 5.710810870780604], [-4.328494218525235, 3.3047586914009837]])

In [9]:
X + Y

MinkowskiSum{Float64,VPolygon{Float64,Array{Float64,1}},VPolygon{Float64,Array{Float64,1}}}(VPolygon{Float64,Array{Float64,1}}(Array{Float64,1}[[-2.891440369388445, 1.4974603483262503], [-2.7543914518160597, 1.0140102595539866], [-2.5160881915505673, 0.4341350873182388], [-1.8326871410944174, 0.2791562864595338], [-1.079324190333, 0.10959707549124292], [-0.5694666832207753, 0.02057642325639991], [0.9206192553020137, -0.18229943741611468], [0.690459131394798, 2.615053718175048], [-2.7090118148649993, 1.78797518892204]]), VPolygon{Float64,Array{Float64,1}}(Array{Float64,1}[[-1.189781063361007, 0.3256034667025641], [-1.1096833678107179, -0.10857070931184043], [-0.31982477529339226, -0.1128606495006116], [1.2135756215119895, -0.07501254151744109], [1.976518461380786, 0.5415029398039025], [2.6540623347530286, 1.3375271945122025], [2.331710899144571, 1.832837203302411], [1.3363698904893284, 2.455481645789108], [0.5346946762643379, 2.4834604152834245], [-1.0131794158530185, 1.0799080389885327

## Faster concrete SIH for hyperrectangular sets

In [1]:
using Revise, LazySets, BenchmarkTools

┌ Info: Precompiling LazySets [b4f0291d-fe17-52bc-9479-3d1a343d9043]
└ @ Base loading.jl:1273


In [16]:
Hr(i) = rand(Hyperrectangle, dim=i)

Hr (generic function with 1 method)

In [6]:
H2 = 
@btime symmetric_interval_hull($H[2])

  158.812 ns (6 allocations: 512 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.0, 0.0], [0.72019121237743, 1.7018467228811314])

In [7]:
# 2D hyperrectangle specialization
function _box_approximation_symmetric_2D(H::Hyperrectangle{N}) where {N}
    @inbounds begin
        # the hyperrectangle is seen as the cartesian product [v₁⁻, v₁⁺] x [v₂⁻, v₂⁺]
        v₁⁺ = abs(H.center[1] + H.radius[1])
        v₁⁻ = abs(H.center[1] - H.radius[1])
        v₂⁺ = abs(H.center[2] + H.radius[2])
        v₂⁻ = abs(H.center[2] - H.radius[2])
    end
    r₁ = max(v₁⁺, v₁⁻)
    r₂ = max(v₂⁺, v₂⁻)
    return Hyperrectangle(zeros(N, 2), [r₁, r₂])
end

_box_approximation_symmetric_2D (generic function with 1 method)

In [10]:
function _box_approximation_symmetric(H::Hyperrectangle{N}) where {N<:Real}
    n = dim(H)
    r = Vector{N}(undef, n)
    @inbounds for i in 1:n
        v⁺ = abs(H.center[i] + H.radius[i])
        v⁻ = abs(H.center[i] - H.radius[i])
        r[i] = max(v⁺, v⁻)
    end
    return Hyperrectangle(zeros(N, n), r)
end

_box_approximation_symmetric (generic function with 1 method)

In [62]:
H2 = Hr(2)
@btime symmetric_interval_hull($H2)
@btime _box_approximation_symmetric_2D($H2)
@btime _box_approximation_symmetric($H2)
@btime _box_approximation_symmetric_opt($H2)

  160.102 ns (6 allocations: 512 bytes)
  58.570 ns (3 allocations: 224 bytes)
  61.320 ns (3 allocations: 224 bytes)
  59.018 ns (3 allocations: 224 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.0, 0.0], [0.2364712197239693, 1.8782676583558344])

In [63]:
H3 = Hr(3)
@btime symmetric_interval_hull($H3)
@btime _box_approximation_symmetric($H3)
@btime _box_approximation_symmetric_opt($H3)

  177.038 ns (6 allocations: 592 bytes)
  64.369 ns (3 allocations: 256 bytes)
  60.445 ns (3 allocations: 256 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.0, 0.0, 0.0], [2.3250612382360627, 1.292439198398232, 0.8485957973177538])

In [64]:
H10 = Hr(10)
@btime symmetric_interval_hull($H10)
@btime _box_approximation_symmetric($H10)
@btime _box_approximation_symmetric_opt($H10)

  361.190 ns (6 allocations: 832 bytes)
  74.784 ns (3 allocations: 352 bytes)
  68.651 ns (3 allocations: 352 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [3.255075464243002, 0.8029571415635, 2.0394287791695196, 1.6496181132295298, 1.7924275314952194, 1.0931244296656248, 0.4199639544549816, 1.6561390010081418, 3.870178311676268, 2.599469197022899])

In [65]:
H20 = Hr(20)
@btime symmetric_interval_hull($H20)
@btime _box_approximation_symmetric($H20)
@btime _box_approximation_symmetric_opt($H20)

  1.258 μs (6 allocations: 1.20 KiB)
  86.183 ns (3 allocations: 512 bytes)
  80.939 ns (3 allocations: 512 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.738511945940129, 1.870078985039739, 0.2953967823033481, 2.8286643289711835, 2.1331375693352777, 0.9331694169289881, 2.3515487921631104, 0.8417973553082323, 1.308921184871655, 2.570528871173411, 1.0770894986438315, 1.276363009372954, 1.2527656605159057, 1.8737706669481047, 1.8052063872937967, 0.5011182354965008, 1.584200191977605, 0.8919888736305867, 0.6390723342286688, 2.107198784222291])

In [66]:
H50 = Hr(50)
@btime symmetric_interval_hull($H50)
@btime _box_approximation_symmetric($H50)
@btime _box_approximation_symmetric_opt($H50)

  5.850 μs (6 allocations: 2.45 KiB)
  125.612 ns (3 allocations: 1.00 KiB)
  105.455 ns (3 allocations: 1.00 KiB)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.6644341581566633, 0.9802657433825532, 0.7155431064413443, 3.0201874798683654, 0.7553461817654328, 1.4327753723997776, 2.368518664301826, 1.85184137958683, 1.6798082684625029, 3.2194671224181954  …  1.133534142987648, 0.9510429945098636, 0.601728718851374, 1.7326886650226352, 0.6991854456349893, 0.43172557483792623, 1.2510793513923935, 0.6607820782224891, 0.8303417763757892, 2.186855308660239])

In [57]:
(120 - 95) / 120

0.20833333333333334

In [67]:
H100 = Hr(100)
@btime symmetric_interval_hull($H100)
@btime _box_approximation_symmetric($H100)
@btime _box_approximation_symmetric_opt($H100)

  18.546 μs (6 allocations: 4.41 KiB)
  199.607 ns (3 allocations: 1.78 KiB)
  158.845 ns (3 allocations: 1.78 KiB)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.881977556914849, 1.955366396009818, 2.8105443151352514, 2.0553168404892492, 1.7345672454230254, 1.5045051820213835, 1.9588190733711932, 2.3674037481792203, 0.6298959225709992, 0.4576553980756949  …  1.1594449120527675, 0.8287620890489426, 0.48158772672307004, 3.444171740249841, 1.6879815825979203, 3.1289944955452915, 1.7187581397638683, 0.8743181086681919, 1.8029836794861513, 0.5432977099788707])

In [68]:
H500 = Hr(500)
@btime symmetric_interval_hull($H500)
@btime _box_approximation_symmetric($H500)
@btime _box_approximation_symmetric_opt($H500)

  373.383 μs (6 allocations: 20.34 KiB)
  985.300 ns (3 allocations: 8.16 KiB)
  870.297 ns (3 allocations: 8.16 KiB)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.680811913485442, 1.5462037343739223, 3.5131368472475417, 3.298206684911397, 1.057794288288392, 0.8866684488213761, 2.312005698628509, 1.5255134778796533, 1.044273091091599, 2.2502784036584056  …  0.23712697005438108, 1.290823260390312, 1.5113645812452494, 1.7222677788916143, 1.9112174649547895, 1.19421816259284, 0.5604751227944353, 0.9750277223289734, 1.6419330996283812, 0.7453368821025871])

In [59]:
(185 - 145) / 185

0.21621621621621623

In [25]:
symmetric_interval_hull(H100) == _box_approximation_symmetric(H100)

true

can we make this faster doing the math..?

In [60]:
@inline function _absmax_interval(c::N, r::N) where {N}
    if c >= zero(N)
        m = c + r
    else
        m = -c + r
    end
    return m
end

function _box_approximation_symmetric_2D_opt(H::Hyperrectangle{N}) where {N}
    r₁ = _absmax_interval(H.center[1], H.radius[1])
    r₂ = _absmax_interval(H.center[2], H.radius[2])
    return Hyperrectangle(zeros(N, 2), [r₁, r₂])
end

function _box_approximation_symmetric_opt(H::Hyperrectangle{N}) where {N<:Real}
    n = dim(H)
    r = Vector{N}(undef, n)
    @inbounds for i in 1:n
        r[i] = _absmax_interval(H.center[i], H.radius[i])
    end
    return Hyperrectangle(zeros(N, n), r)
end

_box_approximation_symmetric_opt (generic function with 1 method)

In [61]:
H2

Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([-0.9074174474655423, 0.6789619152386245], [0.342485177453982, 0.2258403534129026])

In [30]:
@btime _box_approximation_symmetric_2D($H2)

  55.050 ns (3 allocations: 224 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.0, 0.0], [2.43338502503247, 1.80443774883788])

In [31]:
H2.center

2-element Array{Float64,1}:
 1.196705611351988 
 1.1910866133870228

In [32]:
H2.radius

2-element Array{Float64,1}:
 1.236679413680482 
 0.6133511354508572

In [43]:
@btime _box_approximation_symmetric_2D_opt($H2)

  53.765 ns (3 allocations: 224 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.0, 0.0], [2.43338502503247, 1.80443774883788])

In [38]:
c = H2.center[1]

1.196705611351988

In [39]:
r = H2.radius[1]

1.236679413680482

In [49]:
H50 = Hr(50)
@btime symmetric_interval_hull($H50)
@btime _box_approximation_symmetric($H50)
@btime _box_approximation_symmetric_opt($H50)

  5.636 μs (6 allocations: 2.45 KiB)
  118.548 ns (3 allocations: 1.00 KiB)
  94.900 ns (3 allocations: 1.00 KiB)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.9666203297477394, 1.8058815004017976, 1.4592472830152543, 1.2830140312704679, 3.5882924561726464, 1.6821816358458805, 2.598411076771309, 2.8601467024551193, 1.7514792999959394, 4.6061877414632395  …  2.248284040204947, 2.092520646282453, 1.2607303040224784, 2.2165315432187915, 2.5534624350641257, 0.8160513638221383, 1.3986604417345982, 0.9825834776104627, 2.2000074097304627, 1.5943497356243361])

In [50]:
symmetric_interval_hull(H50) == _box_approximation_symmetric_opt(H50)

true

## Similar type and matrix type

In [1]:
using Revise, LazySets, SparseArrays, LinearAlgebra

In [3]:
_vector_type(::Type{<:AbstractSparseArray{T}}) where T = SparseVector{T, Int}
_vector_type(VT::Type{<:AbstractVector{T}}) where T = VT
_vector_type(::Type{<:AbstractMatrix{T}}) where T = Vector{T}

_matrix_type(::Type{<:AbstractVector{T}}) where T = Matrix{T}
_matrix_type(MT::Type{<:AbstractMatrix{T}}) where T = MT
_matrix_type(::Type{<:AbstractSparseVector{T}}) where T = SparseMatrixCSC{T, Int}

_matrix_type (generic function with 3 methods)

In [13]:
# sparse
vec = sprand(10, 0.5)
mat = sprandn(10, 10, 0.5)

@assert _vector_type(typeof(vec)) == SparseVector{Float64, Int}
@assert _matrix_type(typeof(vec)) == SparseMatrixCSC{Float64,Int64}

@assert _vector_type(typeof(mat)) == SparseVector{Float64, Int}
@assert _matrix_type(typeof(mat)) == SparseMatrixCSC{Float64,Int64}

In [26]:
?SparseVector

search: SparseVector AbstractSparseVector sparsevec



```
SparseVector{Tv,Ti<:Integer} <: AbstractSparseVector{Tv,Ti}
```

Vector type for storing sparse vectors.


In [33]:
Vector(sparsevec([1, 3], Float64[1, 3], 3))

3-element Array{Float64,1}:
 1.0
 0.0
 3.0

In [31]:
Matrix(sparse([1, 3], [1, 3], [1.0, 3.0], 3, 3))

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  3.0

In [35]:
N = Float64

Float64

In [36]:
        vec = N[1, 0, 3]
        mat = N[1 0 0; 0 0 0; 0 0 3]

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  3.0

In [14]:
# dense
vec = rand(10)
mat = rand(10, 10)

@assert _vector_type(typeof(vec)) == Vector{Float64}
@assert _matrix_type(typeof(vec)) == Matrix{Float64}

@assert _vector_type(typeof(mat)) == Vector{Float64}
@assert _matrix_type(typeof(mat)) == Matrix{Float64}

In [20]:
# diagonal
mat = Diagonal([1.0, 2.0])

@assert _vector_type(typeof(mat)) == Vector{Float64}
@assert _matrix_type(typeof(mat)) == Diagonal{Float64,Array{Float64,1}}

In [21]:
# other: identity multiple
using MathematicalSystems

mat = I(4, 2)

@assert _vector_type(typeof(mat)) == Vector{Float64}
@assert _matrix_type(typeof(mat)) == IdentityMultiple{Int64}

IdentityMultiple{Int64} of value 4 and order 2

In [23]:
_matrix_type(typeof(mat))

IdentityMultiple{Int64}

In [23]:
t = rand(5)

5-element Array{Float64,1}:
 0.5944166819672283 
 0.36046656032222146
 0.593524092088755  
 0.5686452729365896 
 0.18513535329913444

In [24]:
_vector_type(typeof(t))

Array{Float64,1}

In [30]:
t = rand(2, 2)

2×2 Array{Float64,2}:
 0.879689  0.048161
 0.569673  0.65067 

In [34]:
_matrix_type(typeof(t))

Array{Float64,2}

IdentityMultiple{Int64} of value 4 and order 2

In [28]:
_vector_type(typeof(y))

Array{Int64,1}

In [36]:
_matrix_type(typeof(y))

Array{Int64,2}

In [ ]:
## Zonotope